In [9]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.svm import OneClassSVM 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import itertools

In [2]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 326)
(130, 274)


In [7]:
pca = PCA(n_components=0.95, svd_solver='full')
X_reduced = pca.fit_transform(X)
X_reduced.shape


(130, 5)

In [32]:

total_variance = 0
columns_variance = {}
n_cols = X.shape[1]

for col in X.columns:
    columns_variance[col] = X[col].var()
    total_variance += X[col].var()

current_variance = total_variance
for idx, (col, var) in enumerate(sorted(columns_variance.items(), key=lambda item: item[1])):
    current_variance -= var 
    if (current_variance / total_variance <= 0.95):
        print(n_cols - (idx + 1))
        break

16


In [29]:
def reduce_cols(X, components):
    total_variance = 0
    columns_variance = {}
    n_cols = X.shape[1]

    for col in X.columns:
        columns_variance[col] = X[col].var()
        total_variance += X[col].var()

    cols_to_drop = []
    for idx, (col, var) in enumerate(sorted(columns_variance.items(), key=lambda item: item[1])):
        if (n_cols - (idx + 1) >= components):
            cols_to_drop.append(col) 
        else:
            break

    return X.drop(columns=cols_to_drop)

new_X = reduce_cols(X, 250)
new_X.shape

(130, 250)

In [66]:
def nested_cv(X, components, mod_selection_score=accuracy_score):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo')

    X = reduce_cols(X, components)
    X = X.values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_score = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    score = 0
                    if (mod_selection_score == precision_score):
                        score = mod_selection_score(true_values, pred_values, zero_division=0.0)
                    else:
                        score = mod_selection_score(true_values, pred_values)
                        
                    if score > best_score:
                        best_score = score
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'score used for model selection': mod_selection_score.__name__,
            'method used for model selection': 'nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'components': components}

In [67]:
results = nested_cv(X, 250, accuracy_score)
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250


In [68]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [69]:
scores_df = add_record(scores_df, nested_cv(X, 250, f1_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250


In [70]:
scores_df = add_record(scores_df, nested_cv(X, 250, precision_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250


In [71]:
scores_df = add_record(scores_df, nested_cv(X, 250, recall_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250


In [72]:
scores_df = add_record(scores_df, nested_cv(X, 200, accuracy_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200


In [73]:
scores_df = add_record(scores_df, nested_cv(X, 200, f1_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200


In [74]:
scores_df = add_record(scores_df, nested_cv(X, 200, precision_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200


In [75]:
scores_df = add_record(scores_df, nested_cv(X, 200, recall_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200


In [76]:
scores_df = add_record(scores_df, nested_cv(X, 150, accuracy_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150


In [77]:
scores_df = add_record(scores_df, nested_cv(X, 150, f1_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [78]:
scores_df = add_record(scores_df, nested_cv(X, 150, precision_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [79]:
scores_df = add_record(scores_df, nested_cv(X, 150, recall_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [80]:
scores_df = add_record(scores_df, nested_cv(X, 100, accuracy_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [81]:
scores_df = add_record(scores_df, nested_cv(X, 100, f1_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [82]:
scores_df = add_record(scores_df, nested_cv(X, 100, precision_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [83]:
scores_df = add_record(scores_df, nested_cv(X, 100, recall_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [84]:
scores_df = add_record(scores_df, nested_cv(X, 50, accuracy_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [85]:
scores_df = add_record(scores_df, nested_cv(X, 50, f1_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [86]:
scores_df = add_record(scores_df, nested_cv(X, 50, precision_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [87]:
scores_df = add_record(scores_df, nested_cv(X, 50, recall_score))
scores_df

,algorythm,best kernel,best gamma,best nu,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,components
0,OneClassSVM,rbf,1.000000e-09,0.04,accuracy_score,nested cv,57.307692,9.828409,62.571917,33.690292,30.714286,24.712634,250
1,OneClassSVM,rbf,1.000000e-08,0.12,f1_score,nested cv,59.076923,8.592001,65.499326,25.814174,46.714286,30.646437,250
2,OneClassSVM,linear,1.000000e-09,0.93,precision_score,nested cv,53.692308,9.388398,52.535390,39.560953,24.142857,28.710021,250
3,OneClassSVM,rbf,1.000000e-09,0.28,recall_score,nested cv,56.307692,9.287008,49.242649,28.592036,58.285714,39.314535,250
4,OneClassSVM,rbf,1.000000e-05,0.21,accuracy_score,nested cv,59.153846,11.612328,65.546134,33.760850,33.857143,28.168951,200
5,OneClassSVM,rbf,1.000000e-09,0.48,f1_score,nested cv,61.230769,11.301149,63.711425,30.838550,46.142857,32.014346,200
6,OneClassSVM,linear,1.000000e-09,0.01,precision_score,nested cv,53.076923,7.337994,66.861548,36.492951,19.142857,18.319166,200
7,OneClassSVM,rbf,1.000000e-09,0.22,recall_score,nested cv,59.769231,10.938505,61.711720,31.021775,56.142857,38.439191,200
8,OneClassSVM,rbf,1.000000e-08,0.57,accuracy_score,nested cv,61.615385,11.107064,68.151580,30.472533,41.000000,25.904456,150
9,OneClassSVM,rbf,1.000000e-09,0.40,f1_score,nested cv,63.230769,10.855697,68.447518,25.741636,50.000000,28.320325,150


In [88]:
import pickle

file_path = 'svm_exp2_df.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)